### Učitavanje GenBank fajla i parsiranje zapisa
Fajl `data/NC_000913.3.gb` preuzet sa https://www.ncbi.nlm.nih.gov/nuccore/NC_000913.3?report=gbwithparts&log$=seqview

In [122]:
from Bio import SeqIO
data = SeqIO.parse("data/NC_000913.3.gb", "genbank")

In [123]:
records = [record for record in data]

In [124]:
print(f'Total records: {len(records)}')

Total records: 1


In [125]:
record = records[0]
print(record)

ID: NC_000913.3
Name: NC_000913
Description: Escherichia coli str. K-12 substr. MG1655, complete genome
Database cross-references: BioProject:PRJNA57779, BioSample:SAMN02604091
Number of features: 9285
/molecule_type=DNA
/topology=circular
/data_file_division=CON
/date=17-JUL-2025
/accessions=['NC_000913']
/sequence_version=3
/gi=556503834
/keywords=['RefSeq']
/source=Escherichia coli str. K-12 substr. MG1655
/organism=Escherichia coli str. K-12 substr. MG1655
/taxonomy=['Bacteria', 'Pseudomonadati', 'Pseudomonadota', 'Gammaproteobacteria', 'Enterobacterales', 'Enterobacteriaceae', 'Escherichia']
/references=[Reference(title='Escherichia coli K-12: a cooperatively developed annotation snapshot--2005', ...), Reference(title='Highly accurate genome sequences of Escherichia coli K-12 strains MG1655 and W3110', ...), Reference(title='The complete genome sequence of Escherichia coli K-12', ...), Reference(title='Workshop on Annotation of Escherichia coli K-12', ...), Reference(title='ASAP: 

In [126]:
sequence = record.seq
print(f'Sequence length: {len(sequence)}')

Sequence length: 4641652


In [127]:
CDS = [feature for feature in record.features if feature.type == 'CDS']
print(f'Total CDS: {len(CDS)}')

Total CDS: 4318


### Verifikacija podataka
Provera da li su svi CDS-ovi validni:
- dužina nukleotidnog niza je deljiva sa 3
- koristi se standardna translaciona tabela (11)
- postoji anotirana aminokiselinska sekvenca
- translacija prati striktna CDS pravila (npr. nema unutrašnjih stop kodona)
- prevedena aminokiselinska sekvenca se slaže sa anotiranom

In [131]:
verified_cds = []
corrupted_cds = []
for cds in CDS:
    extracted_cds = cds.extract(record)
    nucleotide_seq = extracted_cds.seq
    nucleotide_sequence_length = len(nucleotide_seq)
    protein_id = cds.qualifiers.get('protein_id', [None])[0]
    cds_locus_tag = cds.qualifiers.get('locus_tag', [None])[0]
    if nucleotide_sequence_length % 3 != 0:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Dužina sekvence {nucleotide_sequence_length} nije deljiva sa 3!')
        continue

    translation_table = cds.qualifiers.get('transl_table', [None])[0]

    if translation_table != '11':
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Ne koristi standardnu translacionu tabelu: {translation_table}')
        continue

    aa_annotated = cds.qualifiers.get('translation', [None])[0]
    if aa_annotated is None:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Ne postoji informacija o sekvenci amino kiseline!')
        continue
    try:
        aa_translated = nucleotide_seq.translate(table=translation_table, to_stop=True, cds=True)
    except Exception as err:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Greska prilikom translacije: {err}')
        continue

    aa_translated = nucleotide_seq.translate(table=translation_table, to_stop=True, cds=True)
    if str(aa_translated) != aa_annotated:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Translacije se ne podudaraju!')
        continue

    verified_cds.append(cds)

Neispravan CDS na lokusu b0240 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b4587 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b4579 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b4580 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b0542 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b0553 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b4570 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b4492 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b1474 za protein NP_415991.1:  Greska prilikom translacije: Extra in frame stop codon 'TGA' found.
Neispravan CDS na lokusu b4582 za protein None:  Ne postoji informacija o sekvenci a

In [132]:
print(f'Ukupan broj ispravnih CDS-ova: {len(verified_cds)}')
print(f'Ukupan broj CDS-ova sa greškom: {len(corrupted_cds)}')

Ukupan broj ispravnih CDS-ova: 4297
Ukupan broj CDS-ova sa greškom: 21
